Notebook para el desarrollo del proyecto datascience desafio latam

### Equipo de trabajo

## Roles para el desarrollo del proyecto

● Líder del equipo                          = Jorge Guerrero / Paulo Gonzalez

● Analista de datos                         = Esteban Sanchez / Gonzalo Rojas

● Control de calidad y validación de datos  = Esteban Sanchez / Gonzalo Rojas

● Ingeniero/a de modelamiento               = Daniel Mardones / Jorge Guerrero

● Visualización de datos                    = Daniel Mardones / Jorge Guerrero

● Documentador/a                            = Daniel Mardoones / Paulo Gonzalez

### Selección del tema y motivación

Se selecciona el tema propuesto Automotora, debido a que es una necesidad replicable, universal y contemporanea a cualquier epoca. Nuestra motivación es generar una herramienta de apoyo a la toma de decisiones de los equipos de compra y venta, para obtener las mejores opciones presentes en el mercado, mejorando la rentabilidad del negocio.

### Planificación de la investigación

 ## Requerimientos:
  
    # 1. Generar una herramienta que ayude al equipo de compras a encontrar las mejores oportunidades.
        Atraves de un modelo de clasificación que nos permita encontrar la mejor rentabilidad, usando técnicas de ML.

    # 2. Definir una propuesta de inversión (lista de automóviles convenientes) para la Automotora.
        Por medio de las metricas obtenidos podremos definir una propuesta para inversion, mediante vehiculos que esten por debajo del precio de venta promedio, lo cual permite un mejor margen de venta.

    # 3. Definir una estrategia de venta por gama de vehículos.
        Se incorpora una columna la cual clasifica la marca del vehiculo en generalista o premium. Con esto podremos generar un analisis del comporatamiento en la venta de vehiculos y con esto desarrollar una estrategia.

    # 4. Encontrar oportunidades de compra y venta entre ciudades/estados.
        Atraves de un EDA, podemos determinar en que ciudades / estados se encuentran las mejores oportunidades de compra y determinar las mejores ciudades / estados para vender estos vehiculos, maximizando la utilidad.

## Hipotesis

    Nuestra hipotesis es poder determinar, mediante un modelo clasificatorio si es rentable la compra de un vehiculo, de acuerdo a sus especifaciones, para una posterior venta.

## Definición del Vector Objetivo

    El vector objetivo es la variable Price.

## Estrategias analíticas a nivel descriptivo

    Se desarrollorá un EDA, el cual nos visualizara la distrubición de vehiculos de acuerdo a la marca, modelos, año, ciudad, estado, millas y valores de vehiculos.

## Modelación y predicción de trabajo

    Como primera estrategia utilizaremos la libreria Lazy Predict, utilizando su implementacion estandar, para conocer de forma preliminar los mejores clasificadores para esta solución. Para almacenar los resultados, utilizaremos una base de datos Postgres, lo que permite obtener mediante una query realizada por web, si es conveniente la compra del vehiculo.

### Disclaimer

- Los datos no seran encriptados.
- Los set de datos no contienen datos faltantes.
- Existen 3 dataset, los cuales por origen se encuentran dividos en: 
    muestra total = true_car_listings.csv
    muestra entrenamiento = true_cars_train.csv
    muestra validacion = true_cars_test.csv
    Debido a esto, no se realiza divisiones de muestra, solo de vector objetivo y matriz de atributos.

In [1]:
# Procesamiento de datos
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import lazypredict


# Machine Learning
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier, RandomForestClassifier 
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import BernoulliNB
from sklearn.metrics import classification_report

from lazypredict.Supervised import LazyClassifier
from lazypredict.Supervised import LazyRegressor


# Tratamiento de warnings
import warnings
warnings.simplefilter('ignore')

In [2]:
df_eda = pd.read_csv('Apoyo Tema Propuesto - Automotora/true_car_listings.csv')
df_eda

,Price,Year,Mileage,City,State,Vin,Make,Model
0,8995,2014,35725,El Paso,TX,19VDE2E53EE000083,Acura,ILX6-Speed
1,10888,2013,19606,Long Island City,NY,19VDE1F52DE012636,Acura,ILX5-Speed
2,8995,2013,48851,El Paso,TX,19VDE2E52DE000025,Acura,ILX6-Speed
3,10999,2014,39922,Windsor,CO,19VDE1F71EE003817,Acura,ILX5-Speed
4,14799,2016,22142,Lindon,UT,19UDE2F32GA001284,Acura,ILXAutomatic
...,...,...,...,...,...,...,...,...
852117,63215,2017,9,Culver City,CA,YV1A22MK9H1013237,Volvo,S90T6
852118,72260,2017,3201,Englewood,NJ,YV4A22PL3H1186162,Volvo,XC90T6
852119,55999,2016,28941,Fort Collins,CO,YV4A22PL4G1000868,Volvo,XC90AWD
852120,60240,2017,3005,San Leandro,CA,YV4A22NLXH1006162,Volvo,V90


In [3]:
df_train = pd.read_csv('Apoyo Tema Propuesto - Automotora/true_cars_train.csv',sep=';' )
df_train

,Price,Year,Mileage,City,State,Vin,Make,Model
0,10888,2013,19606,Long Island City,NY,19VDE1F52DE012636,Acura,ILX5-Speed
1,10999,2014,39922,Windsor,CO,19VDE1F71EE003817,Acura,ILX5-Speed
2,14799,2016,22142,Lindon,UT,19UDE2F32GA001284,Acura,ILXAutomatic
3,7989,2012,105246,Miami,FL,JH4CU2F83CC019895,Acura,TSXAutomatic
4,13995,2013,32384,West Jordan,UT,JH4CU2F64DC006203,Acura,TSX5-Speed
...,...,...,...,...,...,...,...,...
639140,54995,2017,2474,Berwyn,PA,YV1A22MK1H1010445,Volvo,S90T6
639141,72260,2017,3201,Englewood,NJ,YV4A22PL3H1186162,Volvo,XC90T6
639142,55999,2016,28941,Fort Collins,CO,YV4A22PL4G1000868,Volvo,XC90AWD
639143,60240,2017,3005,San Leandro,CA,YV4A22NLXH1006162,Volvo,V90


In [4]:
df_test = pd.read_csv('Apoyo Tema Propuesto - Automotora/true_cars_test.csv',sep=';' )
df_test

,Price,Year,Mileage,City,State,Vin,Make,Model
0,8995,2014,35725,El Paso,TX,19VDE2E53EE000083,Acura,ILX6-Speed
1,8995,2013,48851,El Paso,TX,19VDE2E52DE000025,Acura,ILX6-Speed
2,14490,2014,34032,Greatneck,NY,JH4CU2F84EC002686,Acura,TSXSpecial
3,12921,2012,58550,Boise,ID,JH4CU2F44CC003220,Acura,TSXAutomatic
4,14995,2014,23454,Hackettstown,NJ,19VDE1F31EE009243,Acura,ILX5-Speed
...,...,...,...,...,...,...,...,...
212972,59175,2017,1322,Englewood,NJ,YV1A22MKXH1010413,Volvo,S90T6
212973,54995,2017,9631,Berwyn,PA,YV1A22MK2H1013239,Volvo,S90T6
212974,61990,2017,2201,Winston Salem,NC,YV1A22ML0H1006850,Volvo,S90T6
212975,46500,2017,8,San Leandro,CA,YV140MEM3H1370217,Volvo,V60T5


In [6]:
df_test['Price']

0          8995
1          8995
2         14490
3         12921
4         14995
          ...  
212972    59175
212973    54995
212974    61990
212975    46500
212976    63215
Name: Price, Length: 212977, dtype: int64